# Validalab, s'informer en confiance

Dans ce **TP**, nous allons exploiter la base de données de **Validalab**. <br>
Une organisation à but non lucratif donc le but est d'aider les citoyens à mieux s'informer et à être acteurs de leur consommation d'informations.<br>
Elle a été initiée par **Jean-Marc Guerin**


Plutôt que de faire du fact-checking, **Validalab** prend le parti d'informer les utilisateurs sur la source d'information. <br>
Ainsi, sur cette [application](http://app.validalab.fr/), on peut retrouver différentes informations agrégées sur les médias français.<br>
>Les données ont été scrappées sur différents dites et ingérés dans une base de données **Neo4j**. Le [dictionnaire des données](https://docs.google.com/spreadsheets/d/17iylS3y-xRVZLFMOuyz-o5Oy_MSlO_jvQoY8LBmH1LQ/edit#gid=1217348665) décrit les **types de données**, les **propriétés** et les **relations** du graphe de Validalab.

## Installation du Driver python de Neo4j 

In [ ]:
!pip install neo4j

## Importation des packages 

In [3]:
from neo4j import GraphDatabase, basic_auth

## Informations de connexion à la base de données

In [4]:
password = ""
uri = "neo4j+s://5074307c.databases.neo4j.io"
driver = GraphDatabase.driver(uri,auth=basic_auth("neo4j", password))

In [5]:
db = driver.session(database='neo4j')

### 1- Introduction
Pour exécuter une requête **CYPHER** via le driver python de Neo4j, on utilise la méthode `run` de l'ojbet `Session`, soit `db` dans notre cas.

In [ ]:
# Lister les différents type de données
results = db.run("""
MATCH (n) RETURN DISTINCT labels(n)
""")
results.data()

#### Help
Consultons l'aide de la méthode run.

In [ ]:
help(db.run)

Elle a 2 principaux arguments(`query` et `parameters`) et des arguments indéfinis `**kwargs`.

- `query`: comme l'indique la docstring, c'est tout simplement la requête CYPHER
- `parameters`: il s'agit d'un dictionnaire de paramètres utilisables dans la requête précédente

In [ ]:
# Afficher les informations sur l'entité pertant le nom "Le Monde SA"
results = db.run("""
    MATCH (n:Entity {name: $entity_name})
    RETURN n
    """, {"entity_name": "Le Monde SA"})
results.data()

### 2- Clause MATCH

Combien de nœuds de type **Entity**, **Website**, **Wikipedia** y a-t-il dans la base de données ?
> Créer une fonction permettant de compter le nombre de noeuds pour un Label donné.

**Lien utile :** https://neo4j.com/docs/cypher-manual/current/clauses/match/#basic-node-finding

In [ ]:
def count_nodes(entity_type):
    results = db.run(f"""
    #TO COMPLETE
    """)
    return results.data()[0].get('count')

In [ ]:
for label in ["Entity", "Website", "Wikipedia"]:
    print(f"There is {count_nodes(label)} nodes with the type {label}")

---
Lister les 10 premiers nœuds de type **Entity**

In [ ]:
def find_nodes(entity_type, limit=10):
    results = db.run(f"""
    #TO COMPLETE
    """)
    return results.data()

In [ ]:
find_nodes("Entity", limit=1)

---
Afficher les nœuds de type Entity sous forme de **DataFrame**<br>
**N.B:** la méthode `to_df()` peut aider.

In [ ]:
results = db.run("""
#TO COMPLETE
""")
results.to_df()

### 3- Clause WHERE

Créer une fonction qui permet de retrouver des sites contenant une chaine de caractères.<br>
**Exemple :** lemonde, valeurs, etc...
<br>
**Lien utile :** https://neo4j.com/docs/cypher-manual/current/clauses/where/

In [ ]:
def find_website(string):
    results = db.run(f"""
    #TO COMPLETE
    """)
    return results.data()

In [ ]:
find_website('lemonde')

### 4- RELATIONS

---
Créer une fonction qui retourne le résumé **Wikipedia** pour un site web donné.
> Rappel: Dans la base de données, nous avons les données Wikipedia. Le résumé des noeuds Wikipedia correspond à la propriété ``summary`.

In [ ]:
def get_summary(site_name):
    results = db.run("""
    #TO COMPLETE
    """, {"site_name":site_name})
    return results.data()[0].get('w.summary')

In [ ]:
get_summary('lemessager.fr')

---
Le Gorafi, Le Monde et Valeurs Actuelles sont-ils des journaux fiables ?
Pour répondre à cette question, on peut lister les **citations** de ces médias.<br>
> Les `citations` d'un site, dans notre contexte,  sont les recommandations(<span style="background:green; color:white">positives</span> ou <span style="background:red; color:white">négative</span>) de ce site par des entités quelconques.

In [ ]:
def list_recommendations(site_name):
    results = db.run("""
    #TO COMPLETE
    """, {"site_name":site_name})
    return results.to_df()

In [ ]:
list_recommendations("lemonde.fr")

In [ ]:
list_recommendations("legorafi.fr")

In [ ]:
list_recommendations("valeursactuelles.com")

### 5- Investigations

---
Créer une fonction pour déterminer les propriétaires finaux d'un site web.
> Les propriétairs finaux sont ceux au sommet de la chaine. C'est à dire, ceux qui n'ont personne qui les possède.

In [ ]:
def final_onwers(site_name):
    results = db.run("""
    #TO COMPLETE
    """,
                {"site_name": f"(?i).*{site_name}.*"})
    return [proprio.get('proprietaire') for proprio in results.data()]

In [ ]:
final_onwers('lemessager.fr')

---
Créer une fonction pour déterminer le nombre de médias que possèdent chacun des propriétaires finaux trouvés.

In [ ]:
def medias_by_owners(site_name):
    results = db.run("""
    #TO COMPLETE
    """,
                {"site_name": f"(?i).*{site_name}.*"})
    return results.to_df().sort_values(by="nb_medias", ascending=False)

In [ ]:
medias_by_owners('lemonde.fr')